In [1]:
import os

In [2]:
%pwd

'd:\\SAMITH\\Github\\Air-Quality-Health-Alert-System\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\SAMITH\\Github\\Air-Quality-Health-Alert-System'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    """Data Transformation Configuration"""
    root_dir: Path
    data_path: Path

In [6]:

from Air_Quality_Health_Alert_System.constants import *
from Air_Quality_Health_Alert_System.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config

In [8]:

import pandas as pd
import os
import numpy as np
from Air_Quality_Health_Alert_System import logger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [9]:
class DataTransformation:
    def __init__(self, config):
        self.config = config

    def train_test_spliting(self):
        df = pd.read_csv(self.config.data_path, parse_dates=['date'])  
        df = df.sort_values('date')  

        df.set_index('date', inplace=True) 

        # Impute missing values using interpolation
        df.interpolate(method='time', inplace=True)

        # Select features for modeling
        numeric_features = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
        exclude_cols = ['latitude', 'longitude', 'aqi', 'month', 'day', 'year', 'day_of_week', 
                'hour', 'is_weekend', 'day_of_year']  # keep original
        features_to_scale = [col for col in numeric_features if col not in exclude_cols]

        

        scaler = MinMaxScaler()
        for col in features_to_scale:
            df[col] = df[col].fillna(df[col].mean())

        df[features_to_scale] = scaler.fit_transform(df[features_to_scale])

            
        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(df,test_size = 0.25, random_state = 42)



        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Splitted, cleaned, and over sampled data into training and test sets.")
        logger.info(f"Train shape: {train.shape}")
        logger.info(f"Test shape: {test.shape}")

        print("Train:", train.shape)
        print("Test:", test.shape)


In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2025-08-19 23:39:23,353: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-19 23:39:23,357: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-19 23:39:23,360: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-19 23:39:23,362: INFO: common: created directory at: artifacts]
[2025-08-19 23:39:23,365: INFO: common: created directory at: artifacts/data_transformation]


C:\Users\samit\AppData\Local\Temp\ipykernel_8964\1891697420.py:12: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df.interpolate(method='time', inplace=True)


[2025-08-19 23:39:25,529: INFO: 1891697420: Splitted, cleaned, and over sampled data into training and test sets.]
[2025-08-19 23:39:25,531: INFO: 1891697420: Train shape: (69844, 22)]
[2025-08-19 23:39:25,532: INFO: 1891697420: Test shape: (23282, 22)]
Train: (69844, 22)
Test: (23282, 22)
